In [70]:
import datetime
import json
import os
import sys

from aswe.utils.request import http_request

In [3]:
API_key = os.getenv("SOCCER_API_key")
headers = {"X-Auth-Token": API_key}

In [4]:
def convert_league_name(name):
    valid = {
        "Premier League",
        "Primera Division",
        "Serie A",
        "Bundesliga",
        "Ligue 1",
        "FIFA World Cup",
        "UEFA Champions League",
        "Eredivisie",
        "Campeonato Brasileiro Série A",
        "Championship",
        "Primeira Liga",
        "European Championship",
    }
    if name not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    if name == "Premier League" or name == "English League":
        return "PL"
    elif name == "Primera Division" or name == "Spanish League":
        return "PD"
    elif name == "Serie A" or name == "Italian League":
        return "SA"
    elif name == "Bundesliga" or name == "German League":
        return "BL1"
    elif name == "Ligue 1" or name == "French League":
        return "FL1"
    elif name == "FIFA World Cup" or name == "World Cup":
        return "WC"
    elif name == "UEFA Champions League" or name == "Champions League":
        return "Cl"
    elif name == "Eredivisie" or name == "Dutch League":
        return "DED"
    elif name == "Campeonato Brasileiro Série A" or name == "Brazilian League":
        return "BSA"
    elif name == "Championship" or name == "English League 2":
        return "ELC"
    elif name == "Primeira Liga" or name == "Portuguese League":
        return "PPL"
    elif name == "European Championship" or name == "Euro":
        return "EC"

In [100]:
def get_league_standings(league):
    valid = {"WC", "CL", "BL1", "DED", "BSA", "PD", "FL1", "ELC", "PPL", "EC", "SA", "PL"}
    standings = ""
    if league not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    results = http_request(f"https://api.football-data.org/v4/competitions/{league}/standings", headers=headers).json()
    for team in results["standings"][0]["table"]:
        standings += (
            str(team["position"]) + ". " + team["team"]["name"] + " - " + str(team["points"]) + " points" + "\n"
        )
    return standings

In [101]:
def get_matchday_matches(league, matchday):
    valid = {"WC", "CL", "BL1", "DED", "BSA", "PD", "FL1", "ELC", "PPL", "EC", "SA", "PL"}
    matches = ""
    if league not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    results = http_request(
        f"https://api.football-data.org/v4/competitions/{league}/matches?matchday={matchday}", headers=headers
    ).json()
    for match in results["matches"]:
        if match["status"] == "SCHEDULED" or match["status"] == "TIMED":
            matches += (
                "playing on the "
                + match["utcDate"][-12:-10]
                + "."
                + match["utcDate"][-15:-13]
                + "."
                + match["utcDate"][0:4]
                + " at "
                + str(int(match["utcDate"][-9:-7]) + 2)
                + match["utcDate"][-7:-4]
                + ": "
                + match["homeTeam"]["name"]
                + " vs "
                + match["awayTeam"]["name"]
                + "\n"
            )
        elif match["status"] == "FINISHED":
            matches += (
                "played on the "
                + match["utcDate"][-12:-10]
                + "."
                + match["utcDate"][-15:-13]
                + "."
                + match["utcDate"][0:4]
                + " at "
                + str(int(match["utcDate"][-9:-7]) + 2)
                + match["utcDate"][-7:-4]
                + ": "
                + match["homeTeam"]["name"]
                + " "
                + str(match["score"]["fullTime"]["home"])
                + " : "
                + str(match["score"]["fullTime"]["away"])
                + " "
                + match["awayTeam"]["name"]
                + "\n"
            )
    return matches

In [102]:
def get_ongoing_matches(league=""):
    valid = {"WC", "CL", "BL1", "DED", "BSA", "PD", "FL1", "ELC", "PPL", "EC", "SA", "PL", ""}
    matches = ""
    if league not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    results = http_request(f"https://api.football-data.org/v4/matches?status=IN_PLAY", headers=headers).json()
    for match in results["matches"]:
        if match["competition"]["code"] == league or league == "":
            matches += (
                "playing at "
                + str(int(match["utcDate"][-9:-7]) + 2)
                + match["utcDate"][-7:-4]
                + ": "
                + match["homeTeam"]["name"]
                + " vs "
                + match["awayTeam"]["name"]
                + " in "
                + match["competition"]["name"]
                + "\n"
            )
    if matches == "":
        matches = "No matches are currently being played."
    return matches

In [103]:
def get_matches_today(league=""):
    valid = {"WC", "CL", "BL1", "DED", "BSA", "PD", "FL1", "ELC", "PPL", "EC", "SA", "PL", ""}
    matches = ""
    if league not in valid:
        raise ValueError("results: status must be one of %r." % valid)
    results = http_request(f"https://api.football-data.org/v4/matches?status=SCHEDULED", headers=headers).json()
    for match in results["matches"]:
        if match["competition"]["code"] == league or league == "":
            matches += (
                "playing at "
                + str(int(match["utcDate"][-9:-7]) + 2)
                + match["utcDate"][-7:-4]
                + ": "
                + match["homeTeam"]["name"]
                + " vs "
                + match["awayTeam"]["name"]
                + " in "
                + match["competition"]["name"]
                + "\n"
            )
    return matches